# Suspicious Program Location with Network Connections
Detects programs with network connections running in suspicious files system locations

## Rule Content
```
- title: Suspicious Program Location with Network Connections
  id: 7b434893-c57d-4f41-908d-6a17bf1ae98f
  status: experimental
  description: Detects programs with network connections running in suspicious files
    system locations
  references:
  - https://docs.google.com/spreadsheets/d/17pSTDNpa0sf6pHeRhusvWG6rThciE8CsXTSlDUAZDyo
  author: Florian Roth
  date: 2017/03/19
  logsource:
    product: windows
    service: sysmon
    definition: Use the following config to generate the necessary Event ID 3 Network
      Connection events
    category: null
  detection:
    selection:
      EventID: 3
      Image:
      - '*\$Recycle.bin'
      - '*\Users\All Users\\*'
      - '*\Users\Default\\*'
      - '*\Users\Public\\*'
      - '*\Users\Contacts\\*'
      - '*\Users\Searches\\*'
      - C:\Perflogs\\*
      - '*\config\systemprofile\\*'
      - '*\Windows\Fonts\\*'
      - '*\Windows\IME\\*'
      - '*\Windows\addins\\*'
    condition: selection
  falsepositives:
  - unknown
  level: high

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-endpoint-winevent-sysmon-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(event_id:"3" AND process_path.keyword:(*\\$Recycle.bin OR *\\Users\\All\ Users\\* OR *\\Users\\Default\\* OR *\\Users\\Public\\* OR *\\Users\\Contacts\\* OR *\\Users\\Searches\\* OR C\:\\Perflogs\\* OR *\\config\\systemprofile\\* OR *\\Windows\\Fonts\\* OR *\\Windows\\IME\\* OR *\\Windows\\addins\\*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()